Check each category consist how many data

In [1]:
import pandas as pd

# load the data 
file_path = "Cleaned_News_Articles_Final.csv"
data = pd.read_csv(file_path)
data = data[['text','category']]

data['category'].value_counts()

data.to_csv("original_text_category.csv", index=False)


Category of World and Entertainment aug 1 time with Back Translate technique (Too heavy) - Aborted

In [ ]:
# import pandas as pd
# import nlpaug.augmenter.word as naw
# from tqdm import tqdm
# import random
# import os
# import nltk
# os.environ["TOKENIZERS_PARALLELISM"] = "false"

# # load data
# file_path = "Cleaned_News_Articles_Final.csv"
# data = pd.read_csv(file_path)
# data = data[data['category'].isin(['world', 'entertainment'])]
# data = data[['text', 'category']]

# # minority category
# minority_labels = {
#     "world": 820,
#     "entertainment": 562,
# }

# # using back translate techique to aug
# back_translation_aug = naw.BackTranslationAug(
#     from_model_name='Helsinki-NLP/opus-mt-en-zh',
#     to_model_name='Helsinki-NLP/opus-mt-zh-en')

# # lists to store augmented data
# augmented_summaries = []
# multilabels = []

# for _, row in tqdm(data.iterrows(), total=len(data)):
#     if isinstance(row['category'], str):
#         augmented_labels = row['category'].split(',')  
#         augmented_labels_filtered = [label.strip() for label in augmented_labels if label.strip() in minority_labels]
        
#         if augmented_labels_filtered:
#             # augment the summary for in text
#             augmented_summary_backtranslate = back_translation_aug.augment(row['text'])

#             # append to the lists
#             augmented_summaries.append(augmented_summary_backtranslate)
#             multilabels.append(row['category'])

# augmented_df = pd.DataFrame({'text': augmented_summaries, 'category': multilabels})

# data_augmented_path = 'test.csv'
# augmented_df.to_csv(data_augmented_path, index=False)

Category of World and Entertainment aug 1 time with 1 technique

In [2]:
import pandas as pd
import nlpaug.augmenter.word as naw
from tqdm import tqdm
import random
import os
import nltk
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# load dataset
file_path = "original_text_category.csv"
data = pd.read_csv(file_path)

# minority category
minority_labels = {
    "world": 820,
    "entertainment": 562,
}

# using synonym techique to aug
aug_synonym = naw.SynonymAug(aug_src='wordnet')

# lists to store augmented data
augmented_summaries = []
multilabels = []

for _, row in tqdm(data.iterrows(), total=len(data)):
    if isinstance(row['category'], str):
        augmented_labels = row['category'].split(',')  
        augmented_labels_filtered = [label.strip() for label in augmented_labels if label.strip() in minority_labels]
        
        if augmented_labels_filtered:
            # augment the summary for in text
            augmented_summary_synonym = aug_synonym.augment(row['text'])

            # append to the lists
            augmented_summaries.append(augmented_summary_synonym)
            multilabels.extend([row['category']] * 1)

augmented_df = pd.DataFrame({'text': augmented_summaries, 'category': multilabels})

data_augmented_path = 'aug1.csv'
augmented_df.to_csv(data_augmented_path, index=False)

c:\Users\ChokJoe\anaconda3\envs\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 41795/41795 [00:44<00:00, 933.99it/s]  


Check total of augmented

In [3]:
data_augmented_path = "aug1.csv"
augmented_df = pd.read_csv(data_augmented_path)
augmented_df['category'].value_counts()

category
world            820
entertainment    562
Name: count, dtype: int64

Category of US, Opinions and Waether aug 4 times with 4 different techniques

In [6]:
import pandas as pd
import nlpaug.augmenter.word as naw
from tqdm import tqdm
import random
import os
import nltk
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# load data
file_path = "original_text_category.csv"
data = pd.read_csv(file_path)

# minority category
minority_labels = {
    "us": 317,
    "opinions": 280,
    "weather": 114,
}

# techniques to augmented
aug = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', action="substitute")

aug2 = naw.ContextualWordEmbsAug(
    model_path='roberta-base', action="substitute")

aug_insert = naw.ContextualWordEmbsAug(
    model_path='distilbert-base-uncased', action="insert")

aug_synonym = naw.SynonymAug(aug_src='wordnet')

# lists to store augmented data
augmented_summaries = []
multilabels = []

for _, row in tqdm(data.iterrows(), total=len(data)):
    if isinstance(row['category'], str):
        augmented_labels = row['category'].split(',') 
        augmented_labels_filtered = [label.strip() for label in augmented_labels if label.strip() in minority_labels]
        
        if augmented_labels_filtered:
            # augment the summary for in headline
            augmented_summary = aug.augment(row['text'])
            augmented_summary2 = aug2.augment(row['text'])
            augmented_summary_insert = aug_insert.augment(row['text'])
            augmented_summary_synonym = aug_synonym.augment(row['text'])
            
            # append to the lists
            augmented_summaries.extend([augmented_summary, augmented_summary2, augmented_summary_insert, augmented_summary_synonym])
            multilabels.extend([row['category']] * 4)

augmented_df = pd.DataFrame({'text': augmented_summaries, 'category': multilabels})

data_augmented_path = 'aug2.csv'
augmented_df.to_csv(data_augmented_path, index=False)


100%|██████████| 41795/41795 [4:00:02<00:00,  2.90it/s]   


Check total of augmented

In [8]:
data_augmented_path = "aug2.csv"
augmented_df = pd.read_csv(data_augmented_path)
augmented_df['category'].value_counts()

category
us          1268
opinions    1120
weather      456
Name: count, dtype: int64

Combine both Augmented data

In [9]:
import pandas as pd

# combine data
csv_files = ['aug1.csv', 'aug2.csv']

# initialize an empty list
dfs = []

# append its data to the list
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

combined_data = pd.concat(dfs, ignore_index=True)

final_aug_path = "aug_final.csv"
combined_data.to_csv(final_aug_path, index=False)

Remove square brackets, single quotes, and double quotes

In [10]:
import pandas as pd

input_file = 'aug_final.csv'
output_file = 'aug_final.csv'

# load dataset
df = pd.read_csv(input_file)

# remove square brackets, single quotes, double quotes
df['text'] = df['text'].str.replace(r"[\[\]\"']", '', regex=True)

df.to_csv(output_file, index=False)

Combine Augmented data & without Augmented Data

In [11]:
import pandas as pd

# combine data
csv_files = ['aug_final.csv', 'original_text_category.csv']

# initialize an empty list
dfs = []

# append its data to the list
for file in csv_files:
    df = pd.read_csv(file)
    dfs.append(df)

combined_data = pd.concat(dfs, ignore_index=True)

final_aug_path = "augted_category.csv"
combined_data.to_csv(final_aug_path, index=False)

Downsampling to select each category 1000 data (Balanced)

In [12]:
import pandas as pd

# load dataset
df = pd.read_csv('augted_category.csv')  

# select 10000 data for each sentiment
lower_limit = 100
upper_limit = 1000

# grp the data by sentiment and perform downsampling within each grp
downsampled_data = df.groupby('category').apply(lambda x: x.sample(n=min(upper_limit, max(lower_limit, len(x))), random_state=42))

downsampled_data = downsampled_data.reset_index(drop=True)

downsampled_data.to_csv('balanced_category.csv', index=False)
downsampled_data['category'].value_counts()

C:\Users\ChokJoe\AppData\Local\Temp\ipykernel_23084\2568672057.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  downsampled_data = df.groupby('category').apply(lambda x: x.sample(n=min(upper_limit, max(lower_limit, len(x))), random_state=42))


category
business         1000
entertainment    1000
health           1000
news             1000
opinions         1000
politics         1000
sport            1000
us               1000
world            1000
weather           570
Name: count, dtype: int64